In [5]:
import numpy as np
import TensorFrost as tf

tf.initialize(tf.opengl)

# Householder reflection
def householder_reflection(a):
    v = a.copy()
    v[0] = v[0] + np.copysign(np.linalg.norm(v), a[0])
    v = v / np.linalg.norm(v)
    return np.eye(len(a)) - 2 * np.outer(v, v)

# QR decomposition using Householder reflections
def qr_decomposition(A):
    m, n = A.shape
    Q = np.eye(m)
    R = A.copy()
    for i in range(min(m, n)):
        H = np.eye(m)
        H[i:, i:] = householder_reflection(R[i:, i])
        Q = Q @ H
        R = H @ R
    return Q, R


def modified_gram_schmidt(A):
    """
    Implements the Modified Gram-Schmidt orthogonalization to get the QR decomposition of matrix A.
    A = QR
    """
    A = A.astype(float)  # Ensure A is of float type
    m, n = A.shape
    Q = np.zeros((m, n))
    R = np.zeros((n, n))
    
    for i in range(n-1):
        R[i, i] = np.linalg.norm(A[:, i])
        Q[:, i] = A[:, i] / R[i, i]
        R[i, i+1:n] = np.dot(Q[:, i].T, A[:, i+1:n])
        A[:, i+1:n] -= np.outer(Q[:, i], R[i, i+1:n])
    R[n-1, n-1] = np.linalg.norm(A[:, n-1])
    Q[:, n-1] = A[:, n-1] / R[n-1, n-1]
    return Q, R

#dynamic size QR decomposition
def QRDecomposition():
    A = tf.input([-1, -1], tf.float32)

    m, n = A.shape
    Q = tf.zeros([m, n])
    R = tf.zeros([n, n])
    j = tf.index(0, [m])

    def loop_body(i):
        R[i, i] = tf.norm(A[j, i])
        Q[j, i] = A[j, i] / R[i, i]

        p, k = tf.index_grid([0, i + 1], [m, n])
        t, = tf.index_grid([i+1], [n])
        R[i, t] = tf.sum(Q[p, i] * A[p, k], axis=0)
        A[p, k] -= Q[p, i] * R[i, k]

    tf.loop(loop_body, 0, n-1, 1)

    R[n-1, n-1] = tf.norm(A[j, n-1])
    Q[j, n-1] = A[j, n-1] / R[n-1, n-1]

    return [Q, R]

qr = tf.compile(QRDecomposition)


QRDecomposition:
  Kernel count: 8
  Intermediate buffers: 0
  Host readbacks: 0
  Host writes: 0
  Lines of generated code: 420
  IR Compile time: 1.623700 ms
  Compiler time: 1911.164185 ms



In [2]:
print(qr.list_operations())

List of operations:
int v1_0 = input_shape(cost=0.000000, )
int v1_1 = input_shape(cost=0.000000, )
float m0 = memory(memory_type=input, cost=0.000000, modified, shape=[v1_0,v1_1], )
float m1 = memory(memory_type=output, cost=0.000000, modified, shape=[v1_0,v1_1], )
kernel(cost=0.000000, shape=[v1_0,v1_1], )
{
  int v2_0 = thread_id(cost=0.000000, shape=[v1_0,v1_1], )
  store(memory=[m1], inputs=[v2_1(0.f)], indices=[v2_0], cost=128.000000, shape=[v1_0,v1_1], )
}
float m2 = memory(memory_type=output, cost=0.000000, modified, shape=[v1_1,v1_1], )
kernel(cost=0.000000, shape=[v1_1,v1_1], )
{
  int v4_0 = thread_id(cost=0.000000, shape=[v1_1,v1_1], )
  store(memory=[m2], inputs=[v4_1(0.f)], indices=[v4_0], cost=128.000000, shape=[v1_1,v1_1], )
}
int v5_1 = sub(inputs=[v1_1,v5_0(1)], cost=1.000000, )
int v5_4 = loop(inputs=[v5_2(0),v5_1,v5_3(1)], cost=101.000000, )
{
  kernel(cost=0.000000, shape=[v6_0(1)], )
  {
    int v7_3 = loop(inputs=[v7_2(0),v1_0,v7_1(1)], cost=100.000000, )
    {
 

In [3]:
#generate random matrix
A = np.random.rand(5, 5)

#compute QR decomposition
Q, R = modified_gram_schmidt(A)

#compute QR decomposition using TensorFrost
Atf = tf.tensor(A)
Qtf, Rtf = qr(Atf)
Qnp = Qtf.numpy
Rnp = Rtf.numpy

#check if QR decomposition is correct
print("QR decomposition is correct:", np.allclose(A, np.dot(Q, R)))
print("QR decomposition using TensorFrost is correct:", np.allclose(A, np.dot(Qnp, Rnp)))

#check error
print("Error:", np.linalg.norm(A - np.dot(Q, R)))
print("Error using TensorFrost:", np.linalg.norm(A - np.dot(Qnp, Rnp)))

#print Q and R
print("Q:\n", Qnp)
print("R:\n", Rnp)


QR decomposition is correct: True
QR decomposition using TensorFrost is correct: True
Error: 2.420918330306375e-16
Error using TensorFrost: 1.3874752671600414e-07
Q:
 [[ 0.32278585 -0.44481337  0.7218525   0.410349    0.09215833]
 [ 0.56243694  0.3644122   0.01907808 -0.23873371  0.70250344]
 [ 0.6728503  -0.26141027 -0.16286008 -0.4056528  -0.53652513]
 [ 0.2641761  -0.23418066 -0.6528835   0.65322727  0.14969182]
 [ 0.23865739  0.7390324   0.16057667  0.4282053  -0.43327707]]
R:
 [[ 1.033697    0.75643295  1.0077611   0.6979529   0.9269475 ]
 [ 0.          0.41441217 -0.16519967  0.37530816 -0.01267736]
 [ 0.          0.          0.41954717  0.21928155 -0.2299878 ]
 [ 0.          0.          0.          0.76372916 -0.11893614]
 [ 0.          0.          0.          0.          0.2593881 ]]


In [4]:
#performance test
import time
A = np.random.rand(2000, 2000).astype(np.float32)

#naive NumPy QR decomposition
#start = time.time()
#Q, R = modified_gram_schmidt(A)
#print("Time for naive NumPy QR decomposition:", time.time() - start)

#TensorFrost QR decomposition
Atf = tf.tensor(A)
start = time.time()
Qtf, Rtf = qr(Atf)
print("Time for TensorFrost QR decomposition:", time.time() - start)

#householder QR decomposition
#start = time.time()
#Q, R = qr_decomposition(A)
#print("Time for householder QR decomposition:", time.time() - start)

#built-in NumPy QR decomposition
start = time.time()
Q, R = np.linalg.qr(A)
print("Time for built-in NumPy QR decomposition:", time.time() - start)

print("Error:", np.linalg.norm(A - np.dot(Q, R)))
print("Error using TensorFrost:", np.linalg.norm(A - np.dot(Qtf.numpy, Rtf.numpy)))

Time for TensorFrost QR decomposition: 0.06699585914611816
Time for built-in NumPy QR decomposition: 0.5280008316040039
Error: 0.00045690505
Error using TensorFrost: 0.0005693151
